In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from time import sleep
from  datetime import datetime

In [2]:
biggeek_url = 'https://biggeek.ru/catalog/planshety-apple-ipad'
restore_url = 'https://re-store.ru/apple-ipad/'

In [3]:
# biggeek data

response = requests.get(biggeek_url)
bs = BeautifulSoup(response.text, 'lxml')
page_number = int(bs.findAll('a', class_='prod-pagination__item')[-1].text)

biggeek = []

for page in range(1, page_number+1):

  url = f'{biggeek_url}?page={page}'
  response = requests.get(url)
  bs = BeautifulSoup(response.text, 'lxml')

  sleep(3)
  
  prod = bs.findAll('div', class_='catalog-card')

  for p in prod:
    name = p.find('a', class_='catalog-card__title cart-modal-title').text
    price = p.find('b', class_='cart-modal-count').text

    biggeek.append([name, price])

In [4]:
biggeek_df = pd.DataFrame(biggeek, columns=(['name', 'price']))

biggeek_df['price'] = biggeek_df['price'].str.strip('₽ ').str.replace(' ', '').astype(int)
biggeek_df = biggeek_df[~(biggeek_df['name'].str.contains('Клавиатура')) & ~(biggeek_df['name'].str.contains('Стилус') ) & ~(biggeek_df['name'].str.contains('64'))]
biggeek_df['full_name'] = biggeek_df.loc[:, 'name']
biggeek_df['name'] = biggeek_df['name'].map(lambda x: x[:x.find('(')].strip(' "'))
biggeek_df['year'] = biggeek_df['full_name'].map(lambda x: x[x.find('(')+1 : x.find('(')+5])
biggeek_df['memory'] = biggeek_df['full_name'].map(lambda x: x[x.find(')')+1 : x.lower().find('wi-fi')].strip().replace('GB', 'Gb').replace('TB', 'Tb'))
biggeek_df['wifi_cellular'] = biggeek_df['full_name'].map(lambda x: 
                              x[x.lower().find('wi-fi') : x.lower().find('cellular')+9].strip() 
                              if 'Cellular' in x 
                              else x[x.lower().find('wi-fi') : x.lower().find('wi-fi')+5].strip())
biggeek_df['color'] = biggeek_df['full_name'].map(lambda x: x[x.lower().find('wi-fi')+5:].replace('Cellular', '').strip('+ '))
biggeek_df = biggeek_df[['full_name', 'name', 'year', 'memory', 'wifi_cellular', 'color', 'price']].reset_index(drop=True)

In [5]:
biggeek_df[:5]

,full_name,name,year,memory,wifi_cellular,color,price
0,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi Space ...",Apple iPad Pro 12.9,2022,128Gb,Wi-Fi,Space Gray,98990
1,"Apple iPad Pro 11"" (2022) 128GB Wi-Fi Space Gray",Apple iPad Pro 11,2022,128Gb,Wi-Fi,Space Gray,79990
2,"Apple iPad Pro 11"" (2021) 128Gb Wi-Fi Space Gray",Apple iPad Pro 11,2021,128Gb,Wi-Fi,Space Gray,74990
3,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi Silver",Apple iPad Pro 12.9,2022,128Gb,Wi-Fi,Silver,98990
4,"Apple iPad Pro 11"" (2022) 128GB Wi-Fi Silver",Apple iPad Pro 11,2022,128Gb,Wi-Fi,Silver,79990


In [6]:
# restore data

browser = Chrome('/chromedriver')

browser.get(restore_url)
pages_number = len([int(i.text) for i in browser.find_elements(By.CLASS_NAME, 'pagination__item')])

restore = []
try:
    for page in range(pages_number):
        for prod in browser.find_elements(By.CLASS_NAME, 'catalog__products'):
            restore.append(prod.text.split('\n'))
            browser.find_element(By.CLASS_NAME, 'btn--full-width').click()
            sleep(10)
except NoSuchElementException:
    browser.quit()

/var/folders/hg/nswdp81x16b7__5_j9zd07640000gn/T/ipykernel_19650/4280031027.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome('/chromedriver')


In [7]:
restore_df = []
for i in restore:
    for j in i:
        restore_df.append(j)

In [8]:
restore_df = pd.DataFrame(restore_df, columns=['name'])

restore_df['price'] = restore_df['name'].shift(-1)
restore_df = restore_df[restore_df['name'].str.contains('Apple')].drop_duplicates().reset_index(drop=True)
restore_df['price'] = restore_df['price'].str.strip('₽ ').str.replace(' ', '').astype(int)
restore_df = restore_df[~restore_df['name'].str.contains('64')]
restore_df['full_name'] = restore_df.loc[:, 'name']
restore_df['name'] = restore_df['name'].map(lambda x: x[:x.find('(')].strip(' "').replace(',', '.')) + \
                            restore_df['name'].map(lambda x: x[x.find(')')+1:x.find('"')].replace(',', '.') if 'mini' not in x else '')
restore_df['year'] = restore_df['full_name'].map(lambda x: x[x.find('(')+1 : x.find('(')+5])
restore_df['memory'] = restore_df['full_name'].map(lambda x: x[x.lower().find('wi-fi')+5 : x.find(', ')].replace(' ГБ', 'Gb').replace(' ТБ', 'Tb').strip() 
                            if 'Cellular' not in x 
                            else x[x.lower().find('cellular')+8 : x.find(', ')].replace(' ГБ', 'Gb').replace(' ТБ', 'Tb').strip())
restore_df['wifi_cellular'] = restore_df['full_name'].map(lambda x: 
                              x[x.lower().find('wi-fi') : x.lower().find('cellular')+9].strip() 
                              if 'Cellular' in x 
                              else x[x.lower().find('wi-fi') : x.lower().find('wi-fi')+5].strip())
restore_df['color'] = restore_df['full_name'].map(lambda x: x[x.lower().find('wi-fi')+5:][x[x.lower().find('wi-fi')+5:].find(',')+1:].strip(' «»'))
color_dict = {
    'серый космос':'Space Gray', 'серебристый':'Silver', 
    'синий':'Blue', 'жёлтый':'Yellow', 
    'голубой':'Sky Blue','розовый':'Pink', 
    'сияющая звезда':'Starlight', 'фиолетовый':'Purple'
}
restore_df['color'] = restore_df['color'].map(color_dict)
restore_df = restore_df[['full_name', 'name', 'year', 'memory', 'wifi_cellular', 'color', 'price']].reset_index(drop=True)

In [9]:
restore_df[:5]

,full_name,name,year,memory,wifi_cellular,color,price
0,"Apple iPad Pro (2021) 12,9"" Wi-Fi 128 ГБ, «сер...",Apple iPad Pro 12.9,2021,128Gb,Wi-Fi,Space Gray,99990
1,"Apple iPad 10,2"" (2021) Wi-Fi + Cellular 256 Г...",Apple iPad 10.2,2021,256Gb,Wi-Fi + Cellular,Silver,65990
2,"Apple iPad 10,2"" (2021) Wi-Fi + Cellular 256 Г...",Apple iPad 10.2,2021,256Gb,Wi-Fi + Cellular,Space Gray,65990
3,"Apple iPad 10,2"" (2021) Wi-Fi 256 ГБ, «серый к...",Apple iPad 10.2,2021,256Gb,Wi-Fi,Space Gray,44990
4,"Apple iPad Air (2022) 10,9"" Wi-Fi 256 ГБ, «сер...",Apple iPad Air 10.9,2022,256Gb,Wi-Fi,Space Gray,74990


In [10]:
# main dataframe

df = biggeek_df.merge(restore_df, 
                 how='left',
                 on=['name', 'year', 'memory', 'wifi_cellular', 'color'], 
                 suffixes=['_biggeek', '_restore'])

df['absolute_diff'] = df['price_restore'] - df['price_biggeek']
df['percent_diff'] = round((df['price_restore'] / df['price_biggeek'] - 1) * 100, 2)

df['update_dt'] = datetime.now().strftime('%Y-%m-%d %H:%M')

df = df.sort_values(by='percent_diff', ascending=False)[['full_name_biggeek', 'full_name_restore', 'name', 'year', 
                                                    'color', 'memory', 'wifi_cellular', 'price_biggeek',
                                                    'price_restore', 'absolute_diff', 'percent_diff', 'update_dt']]

In [11]:
df[:30]

,full_name_biggeek,full_name_restore,name,year,color,memory,wifi_cellular,price_biggeek,price_restore,absolute_diff,percent_diff,update_dt
37,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 1...",Apple iPad Pro 12.9,2022,Silver,128Gb,Wi-Fi + Cellular,92990,134990.0,42000.0,45.17,2023-03-12 00:13
30,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 1...",Apple iPad Pro 12.9,2022,Space Gray,128Gb,Wi-Fi + Cellular,92990,134990.0,42000.0,45.17,2023-03-12 00:13
50,"Apple iPad Pro 11"" (2021) 512Gb Wi-Fi Space Gray","Apple iPad Pro (2021) 11"" Wi-Fi 512 ГБ, «серый...",Apple iPad Pro 11,2021,Space Gray,512Gb,Wi-Fi,98990,139990.0,41000.0,41.42,2023-03-12 00:13
60,"Apple iPad Pro 12.9"" (2022) 512GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 5...",Apple iPad Pro 12.9,2022,Space Gray,512Gb,Wi-Fi + Cellular,120990,169990.0,49000.0,40.50,2023-03-12 00:13
97,"Apple iPad Pro 12.9"" (2022) 1TB Wi-Fi + Cellul...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 1...",Apple iPad Pro 12.9,2022,Silver,1Tb,Wi-Fi + Cellular,149990,204990.0,55000.0,36.67,2023-03-12 00:13
55,"Apple iPad Pro 12.9"" (2022) 512GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 5...",Apple iPad Pro 12.9,2022,Silver,512Gb,Wi-Fi + Cellular,124990,169990.0,45000.0,36.00,2023-03-12 00:13
44,"Apple iPad Pro 12.9"" (2022) 256GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 2...",Apple iPad Pro 12.9,2022,Space Gray,256Gb,Wi-Fi + Cellular,110990,144990.0,34000.0,30.63,2023-03-12 00:13
51,"Apple iPad Pro 12.9"" (2022) 256GB Wi-Fi + Cell...","Apple iPad Pro (2022) 12,9"" Wi-Fi + Cellular 2...",Apple iPad Pro 12.9,2022,Silver,256Gb,Wi-Fi + Cellular,110990,144990.0,34000.0,30.63,2023-03-12 00:13
12,"Apple iPad Pro 12.9"" (2022) 256GB Wi-Fi Space ...","Apple iPad Pro (2022) 12,9"" Wi-Fi 256 ГБ, «сер...",Apple iPad Pro 12.9,2022,Space Gray,256Gb,Wi-Fi,102990,129990.0,27000.0,26.22,2023-03-12 00:13
38,"Apple iPad Pro 11"" (2022) 128GB Wi-Fi + Cellul...","Apple iPad Pro (2022) 11"" Wi-Fi + Cellular 128...",Apple iPad Pro 11,2022,Space Gray,128Gb,Wi-Fi + Cellular,83990,104990.0,21000.0,25.00,2023-03-12 00:13
